# Install requried Python modules

In [1]:
# Step 1: Install necessary Python libraries

#!pip install -q -r requirements.txt
!pip install uv


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Use uv to install the dependencies
!uv pip install -r requirements.txt

Using Python 3.11.11 environment at: /opt/app-root
Audited 5 packages in 31ms


# Log in to the OpenShift cluster

In this lab you will interact directly with the OpenShift cluster. To do that you must first log in to the cluster.

In [ ]:
!oc login -u admin -p ${ADMIN_PASSWORD} --server=https://api.${BASE_DOMAIN}:6443

Login successful.

You have access to 109 projects, the list has been suppressed. You can list all projects with 'oc projects'

Using project "vllm-demo".


# 💡 Free up GPU memory to run this exercise

Our `Lab 1 - inference with vllm` is quite computationally intensive. This workshop is only using an L4 GPU so in order ensure the success of jupyterlab notebooks.

We can stop the vLLM inference model servers that are running in the namespace `llama-serving`.

Browse to the Models > Model deployments page in Red Hat OpenShift AI Web Console. **Stop** both the model deployments.

![images/model-serving-stop-start.png](images/model-serving-stop-start.png)

⚠️ **Note:** If you need to redeploy these models then simply start them up again. The **order** you start them up **matters** for correct startup. 

Deploy Llama3 first, then DeepSeek second.

We have limited GPU NVRAM so we use vLLMs `gpu_memory_utilization` parameter when loading the models. This works on available GPU memory so we need to load Llama3 (gpu_memory_utilization=0.5) first then DeepSeek (gpu_memory_utilization=0.8) second.


# 🔍 Environment Validation Script

This script validates the environment by checking:
- vllm version (expected: 0.9.1)
- llmcompressor version (expected: 0.5.2)
- llama-serving namespace status

⚠️ **Note**: You can safely ignore the following "Deprecation Warning" when you run this step.  
*/tmp/ipykernel_1203/152499475.py:3: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html*

In [6]:
import subprocess
import sys
import pkg_resources
from packaging import version
import os

def check_package_version(package_name, expected_version):
    """Check if a package is installed and has the expected version"""
    try:
        installed_version = pkg_resources.get_distribution(package_name).version
        if version.parse(installed_version) >= version.parse(expected_version):
            return True, installed_version
        else:
            return False, installed_version
    except pkg_resources.DistributionNotFound:
        return False, "Not installed"

def check_llama_serving_status():
    """Check the status of llama-serving namespace"""
    try:
        # Check if we can access the namespace
        result = subprocess.run( 
            ["oc", "get", "pods", "-n", "llama-serving", "--no-headers"],
            capture_output=True,
            text=True,
            timeout=30
        )
        
        if result.returncode == 0:
            lines = result.stdout.strip().split('\n')
            if not lines or lines == ['']:
                return True, "No pods found - default vLLM inference serving is disabled (optimal for labs)"
            
            # Check pod status
            pod_status = []
            for line in lines:
                if line.strip():
                    parts = line.split()
                    if len(parts) >= 3:
                        pod_name = parts[0]
                        ready = parts[1]
                        status = parts[2]
                        pod_status.append(f"{pod_name}: {ready} {status}")
            
            return True, pod_status
        else:
            return False, f"Error accessing namespace: {result.stderr}"
    except subprocess.TimeoutExpired:
        return False, "Timeout accessing llama-serving namespace"
    except Exception as e:
        return False, f"Error: {str(e)}"

def validate_environment():
    """Main validation function"""
    print("🔍 Environment Validation Report")
    print("=" * 50)
    
    # Check required packages
    required_packages = {
        "vllm": "0.9.1",
        "llmcompressor": "0.5.2"
    }
    
    all_checks_passed = True
    
    for package, expected_ver in required_packages.items():
        is_valid, installed_ver = check_package_version(package, expected_ver)
        
        if is_valid:
            print(f"✅ {package}: {installed_ver} (>= {expected_ver})")
        else:
            print(f"❌ {package}: {installed_ver} (expected >= {expected_ver})")
            all_checks_passed = False
    
    # Check llama-serving status
    print("\n🔍 llama-serving Status:")
    print("-" * 30)
    
    is_accessible, status_info = check_llama_serving_status()
    
    if is_accessible:
        print("✅ llama-serving namespace is accessible")
        if isinstance(status_info, list):
            for pod_info in status_info:
                print(f"  📦 {pod_info}")
        else:
            print(f"  ℹ️  {status_info}")
    else:
        print(f"❌ llama-serving namespace: {status_info}")
        all_checks_passed = False
    
    print("\n" + "=" * 50)
    if all_checks_passed:
        print("🎉 All validation checks passed! Environment is ready.")
    else:
        print("⚠️  Some validation checks failed. Please review the issues above.")
    
    return all_checks_passed

# Run the validation
validate_environment()

/tmp/ipykernel_961/152499475.py:3: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


🔍 Environment Validation Report
✅ vllm: 0.9.1 (>= 0.9.1)
✅ llmcompressor: 0.5.2 (>= 0.5.2)

🔍 llama-serving Status:
------------------------------
✅ llama-serving namespace is accessible
  📦 llama3-2-3b-predictor-b49576976-6krvj: 0/2 PodInitializing
  📦 sno-deepseek-qwen3-vllm-predictor-694dd7ff5-8nvlm: 1/2 Running

🎉 All validation checks passed! Environment is ready.


True

## Quick Status Check

Run the cell below for a quick status summary:

In [7]:
def quick_status_check():
    """Quick validation check with minimal output"""
    print("🚀 Quick Environment Status Check")
    print("=" * 35)
    
    # Check vllm
    vllm_ok, vllm_ver = check_package_version("vllm", "0.9.1")
    print(f"{'✅' if vllm_ok else '❌'} vllm: {vllm_ver}")
    
    # Check llmcompressor
    llmc_ok, llmc_ver = check_package_version("llmcompressor", "0.5.2")
    print(f"{'✅' if llmc_ok else '❌'} llmcompressor: {llmc_ver}")
    
    # Check llama-serving
    llama_ok, _ = check_llama_serving_status()
    print(f"{'✅' if llama_ok else '❌'} llama-serving: {'accessible, default vLLM inference serving is disabled (optimal for labs)' if llama_ok else 'issue detected'}")
    
    # Overall status
    all_good = vllm_ok and llmc_ok and llama_ok
    print(f"\n{'🎉' if all_good else '⚠️'} Overall: {'All systems ready!' if all_good else 'Issues found - run full validation above'}")
    
    return all_good

# Run quick check
quick_status_check()

🚀 Quick Environment Status Check
✅ vllm: 0.9.1
✅ llmcompressor: 0.5.2
✅ llama-serving: accessible, default vLLM inference serving is disabled (optimal for labs)

🎉 Overall: All systems ready!


True